In [1]:
import pandas as pd
import msgpack
from pathlib import Path

In [2]:
from distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58834 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.00 GiB


In [3]:
from typing import Dict, Any

def cook(rare: Dict[str, Any]) -> Dict[str, Any]:
    same = set(rare["meta"].keys()).intersection(set(rare["alg_model"].keys()))
    for k in same:
        assert rare["meta"][k] == rare["alg_model"][k]

    rare["meta"].update(rare["alg_model"])
    rare.pop("alg_model")
    return rare

DIR = Path("salmon/io/2021-07-01-arr-search")

data = []
for p in DIR.glob("*history.msgpack"):
    raw = msgpack.loads(p.read_bytes(), raw=False)
    data.extend([cook(rare) for rare in raw])

In [4]:
import stats
import targets
from generate_embeddings import _X_test_viz, _X_test
T = targets.get(90)
X_test = _X_test_viz(T)

In [5]:
from toolz import partial
import warnings

def _get_perf(d, T, X_test):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        perf = stats.collect(d["embedding"], T, X_test)
    meta = {f"meta__{k}": v for k, v in d["meta"].items()}
    return {**meta, **perf}
    
T_f = client.scatter(T)
X_test_f = client.scatter(X_test)
futures = [client.submit(_get_perf, d, T=T_f, X_test=X_test_f) for d in data]

In [8]:
from distributed import progress
progress(futures)

VBox()

In [6]:
perfs = client.gather(futures)

In [7]:
df = pd.DataFrame(perfs)

In [9]:
df

,meta__n_responses,meta__time,meta__n,meta__d,meta__R,meta__dataset,meta__random_state,meta__init,meta__max_queries,meta__fname,...,nn_acc_radius_21,nn_acc_radius_22,nn_acc_radius_23,nn_acc_radius_24,nn_acc_radius_25,nn_acc_radius_26,nn_acc_radius_27,nn_acc_radius_28,nn_acc_radius_29,dist_rel_error
0,0,1.625182e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,0.466667,0.466667,0.500000,0.511111,0.511111,0.511111,0.511111,0.522222,0.522222,0.852172
1,48,1.625182e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,0.466667,0.466667,0.500000,0.511111,0.511111,0.511111,0.511111,0.522222,0.522222,0.852172
2,101,1.625182e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,0.433333,0.433333,0.466667,0.466667,0.477778,0.511111,0.533333,0.555556,0.566667,0.869998
3,156,1.625182e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,0.588889,0.611111,0.622222,0.666667,0.700000,0.733333,0.733333,0.733333,0.733333,0.968418
4,205,1.625182e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,0.544444,0.577778,0.588889,0.633333,0.655556,0.677778,0.677778,0.677778,0.700000,0.927096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6767,28482,1.625216e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.937110
6768,28528,1.625216e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.937110
6769,28576,1.625216e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.937110
6770,28625,1.625216e+09,90,2,1,alien_eggs,400,True,30000,{alg}-n_search={n_search}-n_top={n_top}-{respo...,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.937110


In [10]:
import seaborn as sns